<a href="https://colab.research.google.com/github/dmsgustn1938/capstone/blob/main/R_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 워드클라우드

1.   네이버 뉴스 링크 가져오기
2.   네이버 뉴스 양식으로 된 기사 링크 가져오기
3.   제목, 날짜, 본문 내용 가져오기
4.   크롤링 결과 데이터프레임으로 저장 후 csv로 저장

# 네이버 뉴스 링크 가져오기

In [ ]:
library(rvest)
library(dplyr)

In [ ]:
#날짜 입력
year <- '2020'
month <- '08'
day <- '31'

#네이버 뉴스 페이지 링크 가져오기
url <- NULL
 for(x in 0:153){ #뉴스페이지 하나 적게
 url[x+1] <- paste0('https://search.naver.com/search.naver?where=news&sm=tab_jum&query=수소차&ds=' , year , '.' , month, '.01&de=' , year , '.' , month , '.',day,'&news_office_checked=&nso=so%3Ar%2Cp%3Afrom' , year, month, '01to', year , month, day, '&a:all&start=', x*10)
 }

 length(url)

# 네이버 뉴스 양식으로 된 기사 링크 가져오기

In [ ]:
total.link <- NULL

i1 <- 1
i2 <- 1

while(i1 <= length(url)){
    html <- read_html(url[i1])
    link <- html %>% html_nodes('a') %>% html_attr('href')
    link1 <- unique(link)
    link2 <- link1[grep("https://news.naver.com", link1)]
    baselink <- link2[-grep("factcheck", link2)]
        while(i2 <= length(baselink)){
            total.link <- append(total.link, baselink[i2])
            i2 <- i2 + 1
        }      
    i1 <- i1 + 1
    i2 <- 1
}

last.link <-unique(total.link)
length(last.link)

# 제목, 날짜, 본문 내용 가져오기

In [ ]:
total.title <- NULL
total.date <- NULL
total.article <- NULL

i3 <- 1

while(i3 <= length(last.link)) {
    url_new <- last.link[i3]
    base <- read_html(url_new)
    
    #제목
    title <- base %>% html_nodes('#articleTitle') %>% html_text()
    total.title <- append(total.title, title)

    #날짜(수정 전 날짜)
    date <- base %>% html_nodes('.t11') %>% html_text()
    total.date <- append(total.date, date[1])
    
    #본문(공통 불용어 제거)
    article <- base %>% html_nodes('#articleBodyContents') %>% html_text()
    article1 <- gsub("\n","", article)
    article2 <- gsub("\t","", article1)
    article3 <- gsub("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback()","", article2)
    article4 <- gsub("[()]", "", article3)
    article5 <- gsub("[{}]", "", article4)
    total.article <- append(total.article, article5)
    
    i3 <- i3 + 1
}

# 크롤링 결과 데이터프레임으로 저장 후 csv로 저장

In [ ]:
df <- data.frame("제목" = total.title, "날짜" = total.date, "본문" = total.article)
write.csv(df,file = "news.csv")